In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np

In [10]:
def getUniqueModels(allModels):
    ##MUST PASS IN MODEL ENTITY
    ##GETS BEST MODEL FOR EACH INPUT SERIES
    sequences = {} 
    for modelEntity in allModels:
        model = modelEntity["model"]
        thisSeq = str(model.inputSeries.windowSize) + "," + \
                      str(model.inputSeries.series.describe())
        if thisSeq not in sequences:
            sequences[thisSeq] = []
        sequences[thisSeq].append({
            "model":model,
            "info":modelEntity
        })
    modelsToReturn = []
    for seq in sequences:
        bestModel = None
        for modelInfo in sequences[seq]:
            if bestModel is None:
                bestModel = modelInfo
            else:
                if bestModel["info"]["STABILITY"] < modelInfo["info"]["STABILITY"]:
                    bestModel = modelInfo
        modelsToReturn.append(bestModel["model"])
    return modelsToReturn

In [11]:
import portfolio
import dataAck
import warnings
import numpy as np
warnings.filterwarnings("ignore")
tickerToTrade = "EWZ"
allModels = portfolio.getModels(tickerToTrade, returnEntireObject=True)
for modelEntity in allModels:
    model = modelEntity["model"]
    print(model.describe())

((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 5, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 25)
((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 250)
((10, ('GDX', 32, None, None), 5, 'EWZ'), 10, 25)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 2, 'EWZ'), 10, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 250)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)


In [12]:
uniqueModels = getUniqueModels(allModels)
for model in uniqueModels:
    print(model.describe())

TypeError: decoding to str: need a bytes-like object, int found

# VIEW CORRELATION AND COVARIANCE

In [5]:
def generateAllReturns(allModels, joinedData):
    aggregateReturns = None
    aggregatePredictions = None
    for mod in allModels:
        print(mod.describe())
        algoReturn, factorReturn, predictions =  mod.makePredictions(joinedData)
        algoReturn.columns = [str(mod.describe())]
        predictions.columns = [str(mod.describe())]
        if aggregateReturns is None:
            aggregateReturns = algoReturn
            aggregatePredictions = predictions
        else:
            aggregateReturns = aggregateReturns.join(algoReturn)
            aggregatePredictions = aggregatePredictions.join(predictions)
    return aggregateReturns, aggregatePredictions

In [6]:
tickersRequired = []
tickersTraded = []
for mod in uniqueModels:
    
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    

pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
modelReturns, modelPredictions = generateAllReturns(uniqueModels, joinedData)

((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 250)
((22, ('GDX', 32, None, None), 5, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 5, 'EWZ'), 10, 25)
((10, ('GDX', 32, None, None), 2, 'EWZ'), 10, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
GDX
EWZ
((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 250)
THREAD  0 PROGRESS: 0.8333333333333334
THREAD  1 PROGRESS: 0.8333333333333334
THREAD  2 PROGRESS: 0.8333333333333334
THREAD  5 PROGRESS: 0.8333333333333334
THREAD  4 PROGRESS: 0.8333333333333334
THREAD  3 PROGRESS: 0.8333333333333334
THREAD  7 PROGRESS: 0.8333333333333334
THREAD  6 PROGRESS: 0.8333333333333334
THREAD  8 PROGRESS: 0.8333333333333334
THREAD  9 PROGRESS: 0.8333333333333334
THREAD  11 PROGRESS: 0.8333333333333334
THREAD  10 PROGRESS: 0.8333333333333334
THREAD  12 PROGRESS: 0.8333333333333334
THREAD  13 PROGRESS: 0.8333333333333334
THREAD  14 PROGRESS: 0.8333333333333334
THREAD  17 PROGRESS: 0.8333333333333334
THREAD  

Process ForkProcess-276:
Process ForkProcess-278:
Process ForkProcess-295:
Process ForkProcess-300:
Process ForkProcess-290:
Process ForkProcess-260:
Process ForkProcess-296:
Process ForkProcess-251:
Process ForkProcess-293:
Process ForkProcess-297:
Process ForkProcess-268:
Process ForkProcess-252:
Process ForkProcess-259:
Process ForkProcess-284:
Process ForkProcess-262:
Process ForkProcess-257:
Process ForkProcess-294:
Process ForkProcess-303:
Process ForkProcess-269:
Process ForkProcess-302:
Process ForkProcess-254:
Process ForkProcess-298:
Process ForkProcess-299:
Process ForkProcess-281:
Process ForkProcess-292:
Process ForkProcess-285:
Process ForkProcess-301:
Process ForkProcess-265:
Process ForkProcess-263:
Process ForkProcess-275:
Process ForkProcess-267:
Process ForkProcess-279:
Process ForkProcess-291:
Process ForkProcess-270:
Process ForkProcess-288:
Process ForkProcess-286:
Process ForkProcess-255:
Process ForkProcess-256:
Process ForkProcess-304:
Process ForkProcess-264:


KeyboardInterrupt: 

  File "/home/prohb125/walkforwardTrader/dataAck.py", line 304, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(i)+44])
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 286, in runDay
    np.array(targetY),
Traceback (most recent call last):
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 304, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(i)+44])
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 286, in runDay
    np.array(targetY),
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 286, in runDay
    np.array(targetY),
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 304, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(i)+44])
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 286, in runDay
    np.array(targetY),
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 304, in runDayChunking
    pred = self.runDay(xVals[:

In [ ]:
modelReturns

In [ ]:
modelPredictions

In [ ]:
modelReturnsWithFactor = modelReturns.join(dataAck.getDailyFactorReturn(tickerToTrade, joinedData))

In [ ]:
def visualizeModels(modelReturns):
    from string import ascii_letters
    import numpy as np
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    from IPython.display import display

    sns.set(style="white")


    # Compute the correlation matrix
    corr = modelReturns.corr()
    display(corr)

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    from matplotlib.colors import ListedColormap

    # construct cmap
    flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
    cmap = ListedColormap(sns.color_palette(flatui).as_hex())
    
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()
    
    sns.set(style="white")
    
    # Compute the covariance matrix
    cov = modelReturns.cov()
    display(cov)
    # Generate a mask for the upper triangle
    mask = np.zeros_like(cov, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
    cmap = ListedColormap(sns.color_palette(flatui).as_hex())

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(cov, mask=mask, cmap=cmap, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()

In [ ]:
%matplotlib inline
visualizeModels(modelReturnsWithFactor)

# STORE PAST MODEL PREDICTIONS

In [ ]:
def storePastPredictions(allModels, modelPredictions):
    ##THESE ARE SUMMED PREDICTIONS...DIFFERENT THAN PREDICTIONS MADE DAILY
    lastDayUsedPredictions = modelPredictions.dropna()
    for i in range(len(lastDayUsedPredictions.columns)):
        thisModel = allModels[i]
        print(thisModel.describe())
        thisDF = lastDayUsedPredictions[[lastDayUsedPredictions.columns[i]]]
        predictionsToStore = []
        for j in range(len(thisDF.values)):
#             print(thisDF.index[i], thisDF.values[i][0])
            predictionsToStore.append(portfolio.storeAggregateModelPrediction(thisModel, thisDF.values[j][0], thisDF.index[j], shouldReturn=True))
        portfolio.storeManyItems(predictionsToStore)
            

In [ ]:
storePastPredictions(uniqueModels, modelPredictions)

# STORE MODELS AS PORTFOLIO

In [ ]:
def storePortfolio(models, description, benchmark):
    allHashes = []
    for model in models:
        organismHash = hashlib.sha224(str(model.describe()).encode('utf-8')).hexdigest()
        allHashes.append(organismHash)
    
    allHashes.sort()
    portfolioHash = hashlib.sha224(str(allHashes).encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.portfolioDB, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    ##STORE PORTFOLIO OBJECT
    while True:
        try:
            toUpload = {
                "description":description,
                "benchmark":benchmark
            }
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.portfolioLookup, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [ ]:
storePortfolio(uniqueModels, description="Just trade underlying", benchmark=tickerToTrade)

In [49]:
def getPortfolios():
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioLookup)
            retrievedPortfolios = [{
                "key":item.key.name,
                "description":item["description"],
                "benchmark":item["benchmark"]
            } for item in list(query.fetch())]

            return retrievedPortfolios
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [50]:
portfolios = getPortfolios()
print(portfolios)

[{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}]


In [51]:
def getPortfolioModels(portfolioKey):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioDB)
            query.add_filter('portfolio', '=', portfolioKey)
            
            retrievedModels = [item["model"] for item in list(query.fetch())]

            return retrievedModels
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [52]:
for item in portfolios:
    print(item)
    print(getPortfolioModels(item["key"]))

{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
['2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6', '0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3', '11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e', '78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc', '341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9']


In [53]:
def getModelsByKey(modelHashes):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            ##form keys
            keys = []
            for hashing in modelHashes:
                key = datastore_client.key(params.datastoreName, hashing)
                keys.append(key)
                
            retrievedModels = datastore_client.get_multi(keys)
            toReturn = []
            for source in retrievedModels:
                toReturn.append(pickle.loads(source["model"]))
            return toReturn
            
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [54]:
getModelsByKey(getPortfolioModels(item["key"]))